Twin Copoilt using Bayesian Network

In [ ]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Define possible scene types and the features available in each
scene_types = ['Standard', 'Map', 'Virtual_Tour']
features_by_scene = {
    'Standard': ['Model', 'Text', 'Image'],
    'Map': ['Model', 'Text', 'Image', 'GeoJSON'],
    'Virtual_Tour': ['Model', 'Text', 'Image']
}
model_features = ['Translate', 'Measure', 'Rotate', 'Scale']

# Function to generate a random timestamp
def random_timestamp(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

# Function to generate fake data
def generate_data(num_users, num_actions):
    data = []
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)

    for _ in range(num_actions):
        user_id = random.randint(1, num_users)
        scene_type = 'Standard' if random.random() < 0.7 else random.choice(scene_types)

        # Adjusting preferences for 'GeoJSON' and 'Model'
        # When in Map choose GeoJSON 60% of the time.
        if scene_type == 'Map':
            feature_type = 'GeoJSON' if random.random() < 0.6 else random.choice(features_by_scene[scene_type])
        elif scene_type == 'Standard':
            feature_type = 'Model' if random.random() < 0.6 else random.choice(features_by_scene[scene_type])
        elif scene_type == 'Virtual_Tour':
            feature_type = 'Image' if random.random() < 0.9 else random.choice(features_by_scene[scene_type])
        else:
            feature_type = random.choice(features_by_scene[scene_type])

        model_feature = 'Measure' if random.random() < 0.8 else random.choice(model_features)




        timestamp = random_timestamp(start_date, end_date)

        data.append([user_id, scene_type, feature_type, model_feature])

    return pd.DataFrame(data, columns=['User_ID', 'Scene_Type', 'Feature_Type', 'Model_Feature'])

# Generate the data
fake_data = generate_data(10000, 1000000) #10000 Users 1,000,000 choices

# Print the first few rows of the simulated data to verify it looks correct.
print(fake_data.head())


   User_ID Scene_Type Feature_Type Model_Feature
0     3535        Map      GeoJSON       Measure
1      723   Standard        Model       Measure
2     2772   Standard         Text       Measure
3     4922   Standard        Model       Measure
4     2534   Standard        Model       Measure


In [ ]:
import pandas as pd
# Perpare data to fit into Bayesian Network

# Step 1: One-hot encode 'Scene Type' and 'Feature Type'
# This creates a new DataFrame with binary columns for each category
scene_type_encoded = pd.get_dummies(fake_data['Scene_Type'], prefix='Scene')
feature_type_encoded = pd.get_dummies(fake_data['Feature_Type'], prefix='Feature')
model_feature_encoded = pd.get_dummies(fake_data['Model_Feature'], prefix='Model')

# Step 2: Combine the one-hot encoded columns back into a single DataFrame
simulated_data = pd.concat([scene_type_encoded, feature_type_encoded, model_feature_encoded], axis=1)

# Step 3: Verify the structure
print(simulated_data.head())

   Scene_Map  Scene_Standard  Scene_Virtual_Tour  Feature_GeoJSON  \
0          1               0                   0                1   
1          0               1                   0                0   
2          0               1                   0                0   
3          0               1                   0                0   
4          0               1                   0                0   

   Feature_Image  Feature_Model  Feature_Text  Model_Measure  Model_Rotate  \
0              0              0             0              1             0   
1              0              1             0              1             0   
2              0              0             1              1             0   
3              0              1             0              1             0   
4              0              1             0              1             0   

   Model_Scale  Model_Translate  
0            0                0  
1            0                0  
2            0

In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD
from sklearn.model_selection import train_test_split

import numpy as np
from pgmpy.sampling import BayesianModelInference, _return_samples
from pgmpy.utils.mathext import sample_discrete, sample_discrete_maps
import itertools
from collections import defaultdict, namedtuple
import networkx as nx
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from pgmpy.factors import factor_product

# Bayesian Network structure
bayesNet = BayesianNetwork()

bayesNet.add_node("Scene_Standard")
bayesNet.add_node("Scene_Map")
bayesNet.add_node("Scene_Virtual_Tour")

bayesNet.add_node("Feature_Model")
bayesNet.add_node("Feature_Text")
bayesNet.add_node("Feature_Image")
bayesNet.add_node("Feature_GeoJSON")

bayesNet.add_node("Model_Translate")
bayesNet.add_node("Model_Measure")
bayesNet.add_node("Model_Rotate")
bayesNet.add_node("Model_Scale")

bayesNet.add_edge("Scene_Standard", "Feature_Model")
bayesNet.add_edge("Scene_Map", "Feature_Model")
bayesNet.add_edge("Scene_Virtual_Tour", "Feature_Model")

bayesNet.add_edge("Scene_Standard", "Feature_Text")
bayesNet.add_edge("Scene_Map", "Feature_Text")
bayesNet.add_edge("Scene_Virtual_Tour", "Feature_Text")

bayesNet.add_edge("Scene_Standard", "Feature_Image")
bayesNet.add_edge("Scene_Map", "Feature_Image")
bayesNet.add_edge("Scene_Virtual_Tour", "Feature_Image")

bayesNet.add_edge("Scene_Map", "Feature_GeoJSON")

bayesNet.add_edge("Feature_Model", "Model_Translate")
bayesNet.add_edge("Feature_Model", "Model_Measure")
bayesNet.add_edge("Feature_Model", "Model_Rotate")
bayesNet.add_edge("Feature_Model", "Model_Scale")

# Split Data for Training and for Testing 80/20
train_data, test_data = train_test_split(simulated_data, test_size=0.2, random_state=42)

# Fit the Bayesian Network with the new data (likelyhood of different states given parents)
# learn the conditional probability distributions (CPDs) from your training data
bayesNet.fit(train_data, estimator=MaximumLikelihoodEstimator)

# # Print the learned CPDs
for cpd in bayesNet.get_cpds():
    print(f"CPD of {cpd.variable}:")
    print(cpd)



CPD of Scene_Standard:
+-------------------+----------+
| Scene_Standard(0) | 0.200556 |
+-------------------+----------+
| Scene_Standard(1) | 0.799444 |
+-------------------+----------+
CPD of Scene_Map:
+--------------+----------+
| Scene_Map(0) | 0.899981 |
+--------------+----------+
| Scene_Map(1) | 0.100019 |
+--------------+----------+
CPD of Scene_Virtual_Tour:
+-----------------------+----------+
| Scene_Virtual_Tour(0) | 0.899463 |
+-----------------------+----------+
| Scene_Virtual_Tour(1) | 0.100538 |
+-----------------------+----------+
CPD of Feature_Model:
+--------------------+-----------------------+-----+-----------------------+
| Scene_Map          | Scene_Map(0)          | ... | Scene_Map(1)          |
+--------------------+-----------------------+-----+-----------------------+
| Scene_Standard     | Scene_Standard(0)     | ... | Scene_Standard(1)     |
+--------------------+-----------------------+-----+-----------------------+
| Scene_Virtual_Tour | Scene_Virtua

In [ ]:
bayesNet.check_model()
print("Model is correct.")

Model is correct.


In [ ]:
solver = VariableElimination(bayesNet)
# also seen above
feature_model_dist = solver.query(variables=['Feature_GeoJSON'], evidence={'Scene_Map': 1})
print("Probability Distribution for GeoJSON given Map Scene=1:")
print(feature_model_dist)

model_translate_dist = solver.query(variables=['Feature_Model'], evidence={'Scene_Standard': 1})
print("Probability Distribution for Feature_Model given Scene_Standard=1:")
print(model_translate_dist)

model_translate_dist = solver.query(variables=['Model_Measure'], evidence={'Feature_Model': 1})
print("Probability Distribution for Model_Measure given Feature_Model=1:")
print(model_translate_dist)

Probability Distribution for GeoJSON given Map Scene=1:
+--------------------+------------------------+
| Feature_GeoJSON    |   phi(Feature_GeoJSON) |
+====================+========================+
| Feature_GeoJSON(0) |                 0.3003 |
+--------------------+------------------------+
| Feature_GeoJSON(1) |                 0.6997 |
+--------------------+------------------------+
Probability Distribution for Feature_Model given Scene_Standard=1:
+------------------+----------------------+
| Feature_Model    |   phi(Feature_Model) |
+==================+======================+
| Feature_Model(0) |               0.3119 |
+------------------+----------------------+
| Feature_Model(1) |               0.6881 |
+------------------+----------------------+
Probability Distribution for Model_Measure given Feature_Model=1:
+------------------+----------------------+
| Model_Measure    |   phi(Model_Measure) |
+==================+======================+
| Model_Measure(0) |               

In [ ]:
# Independencies
independencies = bayesNet.local_independencies(['Scene_Standard', 'Scene_Map', 'Scene_Virtual_Tour',
                                                'Feature_Model', 'Feature_Text', 'Feature_Image',
                                                'Feature_GeoJSON', 'Model_Translate', 'Model_Measure',
                                                'Model_Rotate', 'Model_Scale'])
print(independencies)




(Scene_Standard ⟂ Scene_Map, Scene_Virtual_Tour, Feature_GeoJSON)
(Scene_Map ⟂ Scene_Virtual_Tour, Scene_Standard)
(Scene_Virtual_Tour ⟂ Scene_Map, Scene_Standard, Feature_GeoJSON)
(Feature_Model ⟂ Feature_Text, Feature_Image, Feature_GeoJSON | Scene_Map, Scene_Virtual_Tour, Scene_Standard)
(Feature_Text ⟂ Feature_Model, Model_Scale, Model_Translate, Model_Rotate, Feature_Image, Model_Measure, Feature_GeoJSON | Scene_Map, Scene_Virtual_Tour, Scene_Standard)
(Feature_Image ⟂ Feature_Model, Model_Scale, Model_Translate, Feature_Text, Model_Rotate, Model_Measure, Feature_GeoJSON | Scene_Map, Scene_Virtual_Tour, Scene_Standard)
(Feature_GeoJSON ⟂ Model_Translate, Feature_Text, Feature_Image, Model_Measure, Feature_Model, Model_Scale, Scene_Virtual_Tour, Model_Rotate, Scene_Standard | Scene_Map)
(Model_Translate ⟂ Scene_Map, Feature_Text, Feature_Image, Model_Measure, Model_Scale, Scene_Virtual_Tour, Model_Rotate, Scene_Standard, Feature_GeoJSON | Feature_Model)
(Model_Measure ⟂ Scene_Map, 

In [ ]:
from sklearn.metrics import accuracy_score

# Initialize the Variable Elimination solver
inference = VariableElimination(bayesNet)

# Evaluate test data vs perdictions
# Prob GeoJSON given Map
predictions = []
actuals = []

for index, row in test_data.iterrows():
    actual_value = row['Feature_GeoJSON']
    actuals.append(actual_value)

    # Extract the evidence for 'GeoJSON' from its parent node 'Scene_Map' (true or false)
    evidence = {'Scene_Map': row['Scene_Map']}

    # Query the Bayesian network to get the probability distribution for 'GeoJSON'
    predicted_distribution = inference.query(variables=['Feature_GeoJSON'], evidence=evidence)

    # Convert the predicted probability distribution to a single predicted state
    predicted_state = np.argmax(predicted_distribution.values)
    predictions.append(predicted_state)

accuracy = accuracy_score(actuals, predictions)

print(f"Accuracy of the Bayesian network on the test set for P(GeoJSON given Map): {accuracy}")

Accuracy of the Bayesian network on the test set for P(GeoJSON given Map): 0.96982


In [ ]:
# Prob Measurement given Model
predictions = []
actuals = []

for index, row in test_data.iterrows():
    actual_value = row['Model_Measure']
    actuals.append(actual_value)

    # Extract the evidence for 'GeoJSON' from its parent node 'Scene_Map' (true or false)
    evidence = {'Feature_Model': row['Feature_Model']}

    # Query the Bayesian network to get the probability distribution for 'GeoJSON'
    predicted_distribution = inference.query(variables=['Model_Measure'], evidence=evidence)

    # Convert the predicted probability distribution to a single predicted state
    predicted_state = np.argmax(predicted_distribution.values)
    predictions.append(predicted_state)

accuracy = accuracy_score(actuals, predictions)

print(f"Accuracy of the Bayesian network on the test set for P(Measure given Model): {accuracy}")

Accuracy of the Bayesian network on the test set for P(Measure given Model): 0.851155


In [ ]:
# Prob Model given Standard
predictions = []
actuals = []

for index, row in test_data.iterrows():
    actual_value = row['Feature_Model']
    actuals.append(actual_value)

    # Extract the evidence for 'GeoJSON' from its parent node 'Scene_Map' (true or false)
    evidence = {'Scene_Standard': row['Scene_Standard']}

    # Query the Bayesian network to get the probability distribution for 'GeoJSON'
    predicted_distribution = inference.query(variables=['Feature_Model'], evidence=evidence)

    # Convert the predicted probability distribution to a single predicted state
    predicted_state = np.argmax(predicted_distribution.values)
    predictions.append(predicted_state)

accuracy = accuracy_score(actuals, predictions)

print(f"Accuracy of the Bayesian network on the test set for P(Model given Standard): {accuracy}")

Accuracy of the Bayesian network on the test set for P(Model given Standard): 0.77511


In [ ]:
# Prob Measure given Standard
predictions = []
actuals = []

for index, row in test_data.iterrows():
    actual_value = row['Model_Measure']
    actuals.append(actual_value)

    # Extract the evidence for 'GeoJSON' from its parent node 'Scene_Map' (true or false)
    evidence = {'Scene_Standard': row['Scene_Standard']}

    # Query the Bayesian network to get the probability distribution for 'GeoJSON'
    predicted_distribution = inference.query(variables=['Model_Measure'], evidence=evidence)

    # Convert the predicted probability distribution to a single predicted state
    predicted_state = np.argmax(predicted_distribution.values)
    predictions.append(predicted_state)

accuracy = accuracy_score(actuals, predictions)

print(f"Accuracy of the Bayesian network on the test set for P(Measure given Standard): {accuracy}")

Accuracy of the Bayesian network on the test set for P(Measure given Standard): 0.849745
